In [1]:
from sklearn.preprocessing import OneHotEncoder
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import sys
import re

import numpy as np
import pandas as pd
import plotly.plotly as py
from plotly.graph_objs import *


from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

pd.options.display.max_columns=999

%matplotlib inline

In [2]:
def add_datepart(df, fldname, drop=True, time=False):
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.
    Examples:
    ---------
    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df
        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13
    >>> add_datepart(df, 'A')
    >>> df
        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    """
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear','Hour']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [21]:
tweets = pd.read_csv('./tweets.csv')
users = pd.read_csv('./users.csv')
# regular_reader_tweets = pd.read_csv('./regular_reader_tweets.csv')

#merge the data
# fulltweets = tweets.merge(users, how='left', left_on='user_id', right_on='id')
fulltweets = pd.read_csv('fulltweets2.csv')

# add_datepart(fulltweets, 'created_str')

In [22]:
fulltweets

,user_id,user_key,created_str,retweet_count,retweeted,favorite_count,text,source,hashtags,posted,mentions,id,location,name,followers_count,statuses_count,time_zone,verified,lang,screen_name,description,created_at,favourites_count,friends_count,listed_count,created_strYear,created_strMonth,created_strDay,created_strDayofweek,created_strDayofyear,created_strHour,created_strMinute,created_strElapsed,Sentiment,Subjectivity
0,2.532612e+09,kathiemrr,2017-02-27 14:54:00,NaN,NaN,NaN,thingsdonebymistake kissing auntie lips,NaN,"[""ThingsDoneByMistake""]",POSTED,[],2.532612e+09,Atlanta,Kathie,2970.0,4267.0,Eastern Time (US & Canada),False,en,KathieMrr,"Imperfection is beauty, madness is genius and ...",Thu May 29 12:50:04 +0000 2014,2094.0,3157.0,22.0,2017,2,27,0,58,14,54,1488207240,0.000000,0.000000
1,2.531160e+09,traceyhappymom,2016-08-15 14:50:20,NaN,NaN,NaN,derpin theolderweget pessimistic httpstcozs3jh...,NaN,"[""TheOlderWeGet""]",POSTED,[],2.531160e+09,"Washington, DC",Tracy Brooks,3696.0,4655.0,Eastern Time (US & Canada),False,en,traceyhappymom,Great thoughts speak only to the thoughtful mi...,Mon May 05 14:13:47 +0000 2014,3778.0,3353.0,21.0,2016,8,15,0,228,14,50,1471272620,0.000000,0.000000
2,1.694026e+09,jacquelinisbest,2016-09-18 19:46:25,0.0,False,0.0,nahbabynah twitchy chuck todd caught shilling ...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","[""WakeUpAmerica""]",POSTED,"[""nahbabynah""]",1.694026e+09,United States,Jackie Best,2078.0,3416.0,Eastern Time (US & Canada),False,en,JacquelinIsBest,"I love God,my family & politics. Conservative....",Fri Aug 23 14:41:32 +0000 2013,3518.0,1757.0,31.0,2016,9,18,6,262,19,46,1474227985,0.000000,0.000000
3,2.587101e+09,judelambertusa,2016-04-07 11:37:45,NaN,NaN,NaN,mcicero10 berniesanders trump people rally tog...,NaN,"[""BernieSanders""]",POSTED,[],2.587101e+09,USA,Jude Lambert,1688.0,1515.0,Central Time (US & Canada),False,en,JudeLambertUSA,God. Country. Family. #tcot\n#RedNationRising ...,Fri Jun 06 13:58:27 +0000 2014,939.0,1525.0,21.0,2016,4,7,3,98,11,37,1460029065,0.000000,0.000000
4,1.647046e+09,carriethornthon,2016-10-10 21:04:06,0.0,False,0.0,itsjustjaynie hillaryclinton undecided voters ...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],POSTED,"[""itsjustjaynie"",""hillaryclinton"",""cnn""]",1.647046e+09,Alaska,Carrie,2291.0,3222.0,Eastern Time (US & Canada),False,en,CarrieThornthon,Conservative Christian Momma. Political junkie...,Mon Aug 05 05:56:27 +0000 2013,3930.0,2272.0,23.0,2016,10,10,0,284,21,4,1476133446,0.000000,0.000000
5,1.657754e+09,johnbranchh,2015-11-11 23:54:42,NaN,NaN,NaN,todaycleveland,NaN,[],POSTED,[],1.657754e+09,"Houston, TX",Jonathan Branch,849.0,1685.0,Eastern Time (US & Canada),False,en,JohnBranchh,Your biggest fear should be the fact I'm Black...,Fri Aug 09 13:11:39 +0000 2013,2359.0,852.0,7.0,2015,11,11,2,315,23,54,1447286082,0.000000,0.000000
6,2.620870e+09,puredavie,2016-12-21 14:35:32,NaN,NaN,NaN,resolution my4wordnewyearsresolution,NaN,"[""My4WordNewYearsResolution""]",POSTED,[],2.620870e+09,USA,David Collier,1780.0,1757.0,Eastern Time (US & Canada),False,en,pureDavie,I would rather die of passion than of boredom.,Wed Jun 18 19:34:30 +0000 2014,1055.0,1603.0,14.0,2016,12,21,2,356,14,35,1482330932,0.000000,0.000000
7,1.657695e+09,daileyjadon,2016-11-21 11:04:00,NaN,NaN,NaN,lifetime movie psycho neighbor digging grave r...,NaN,"[""TVLifeLessonsILearned""]",POSTED,[],1.657695e+09,US,Jadon Dailey,1753.0,2299.0,Eastern Time (US & Canada),False,en,DaileyJadon,"You only live once, but if you do it right, on...",Fri Aug 09 12:38:08 +0000 2013,1298.0,1485.0,13.0,2016,11,21,0,326,11,4,1479726240,0.000000,0.000000
8,2.533654e+09,evagreen69,2017-01-20 02:16:36,NaN,NaN,NaN,conservatexian post alarm alshababs force soma...,NaN,[],POSTED,[],2.533654e+09,USA,Eva Green,2668.0,1831.0,Eastern Time (US & Canada),False,en,EvaGreen69,"God comes 1st. My Daughters, & my husband reti...",Thu May 29 23:40:26 +0000 2014,1325.0,2884.0,46.0,2017,1,20,

In [54]:
subusers = fulltweets.groupby('user_key', as_index=False).mean().sort_values(by='followers_count', ascending=False).user_key[0:20].values

In [56]:
subfulltweets2 = fulltweets[fulltweets.user_key.isin(subusers)][['user_key', 'followers_count', 'Subjectivity', 'Sentiment', 'statuses_count', 'created_str']]

In [58]:
add_datepart(subfulltweets2, 'created_str')

In [61]:
data2015 = subfulltweets2[subfulltweets2.created_strYear==2015]
data2016 = subfulltweets2[subfulltweets2.created_strYear==2016]
data2017 = subfulltweets2[subfulltweets2.created_strYear==2017]

In [31]:
data2015.shape,data2016.shape,data2017.shape


((162, 13), (8938, 13), (1614, 13))

In [9]:
days = {
    0:'Sunday',
    1:'Monday',
    2:'Tuesday',
    3:'Wednesday',
    4:'Thursday',
    5:'Friday',
    6:'Saturday'
}
# data.created_strDayofweek = data.created_strDayofweek.map(days)

In [65]:
data2015

,user_key,followers_count,Subjectivity,Sentiment,statuses_count,created_strYear,created_strMonth,created_strWeek,created_strDay,created_strDayofweek,created_strDayofyear,created_strHour,created_strElapsed
1982,politweecs,32846.0,0.200000,0.050,559.0,2015,5,18,2,5,122,23,1430608666
2029,jenn_abrams,61609.0,0.000000,0.000,20576.0,2015,12,49,5,5,339,16,1449331239
4972,giselleevns,24344.0,0.000000,0.000,18466.0,2015,12,50,8,1,342,14,1449586075
5021,jenn_abrams,61609.0,0.166667,0.125,20576.0,2015,5,20,12,1,132,18,1431454492
5187,jenn_abrams,61609.0,0.600000,-0.100,20576.0,2015,6,25,17,2,168,17,1434563399
5764,jenn_abrams,61609.0,0.000000,0.000,20576.0,2015,5,19,4,0,124,22,1430777773
5937,politweecs,32846.0,0.000000,0.000,559.0,2015,8,34,19,2,231,9,1439975481
8039,politweecs,32846.0,0.350000,0.350,559.0,2015,4,16,17,4,107,20,1429303801
9439,jenn_abrams,61609.0,0.000000,0.000,20576.0,2015,7,29,14,1,195,15,1436886573
9581,jenn_abrams,61609.0,0.000000,0.000,20576.0,2015,7,29,14,1,195,17,1436894165


In [107]:
m2015 =pd.pivot_table(data2015,values = 'user_key',index='created_strDayofweek',
                      columns='created_strMonth', aggfunc=len,fill_value=0, dropna=False)
m2016 =pd.pivot_table(data2016,values = 'user_key',index='created_strDayofweek',
                      columns='created_strMonth', aggfunc=len,fill_value=0, dropna=False)
m2017 =pd.pivot_table(data2017,values = 'user_key',index='created_strDayofweek',
                      columns='created_strMonth', aggfunc=len,fill_value=0, dropna=False)

z2015=m2015.as_matrix()
z2016=m2016.as_matrix()
z2017=m2017.as_matrix()

In [79]:
m2015

created_strMonth,1,2,3,4,5,6,7,8,9,10,11,12
created_strDayofweek,,,,,,,,,,,,
0,0,0,0,0,1,3,2,1,2,1,1,4
1,0,0,0,1,3,5,37,3,0,0,0,1
2,0,0,0,1,0,1,8,6,2,0,1,0
3,0,1,0,2,0,4,1,9,1,0,0,1
4,0,0,0,3,3,3,2,2,0,0,0,2
5,1,0,1,7,14,1,3,1,0,0,0,1
6,0,0,1,1,0,1,5,4,3,0,0,0


In [108]:
trace = Heatmap(z=z2015,
                x=[i for i in np.arange(0,13)],
                y=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday'],
                colorscale = 'Jet'
               )
layout = Layout(
    title='Number of Tweets during the day of the week per hour',
    xaxis=dict(nticks=24)

)
data=[trace]
fig = Figure(data=data,layout=layout)
# plot(fig, filename='numberoftweets.html')
iplot(fig, filename='numberoftweets.html')

In [109]:
trace = Heatmap(z=z2016,
                x=[i for i in np.arange(0,13)],
                y=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday'],
                colorscale = 'Jet'
               )
layout = Layout(
    title='Number of Tweets during the day of the week per hour',
    xaxis=dict(nticks=24)

)
data=[trace]
fig = Figure(data=data,layout=layout)
# plot(fig, filename='numberoftweets.html')
iplot(fig, filename='numberoftweets.html')

In [110]:
trace = Heatmap(z=z2017,
                x=[i for i in np.arange(0,5)],
                y=['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday'],
                colorscale = 'Jet'
               )
layout = Layout(
    title='Number of Tweets during the day of the week per hour'
)
data=[trace]
fig = Figure(data=data,layout=layout)
# plot(fig, filename='numberoftweets.html')
iplot(fig, filename='numberoftweets.html')

In [87]:
data2017.created_strMonth.unique()

array([1, 2, 3, 4])

In [95]:
m=pd.pivot_table(subfulltweets2,values = 'user_key',index='created_strYear',
                      columns='created_strMonth', aggfunc=len,fill_value=0, dropna=False)

z=m.as_matrix()
m

created_strMonth,1,2,3,4,5,6,7,8,9,10,11,12
created_strYear,,,,,,,,,,,,
2015,1,1,2,15,21,18,58,26,8,1,2,9
2016,1,1,8,5,34,27,391,1184,1192,3051,1354,1690
2017,1476,127,10,1,0,0,0,0,0,0,0,0


In [111]:
trace = Heatmap(z=z,
                x=[str(i) for i in np.arange(0,13)],
                y=['2015','2016','2017'],
                colorscale = 'Jet'
               )
layout = Layout(
    title='Number of Tweets during the day of the week per hour',
    xaxis=dict(nticks=12)

)
data=[trace]
fig = Figure(data=data,layout=layout)
# plot(fig, filename='numberoftweets.html')
iplot(fig, filename='numberoftweets.html')